In [1]:
# pip install 
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
import time
from scipy.stats import linregress
import seaborn as sns

#from api_keys import api_key
from citipy import citipy

In [2]:
csvpath = "../gt-prj1-crime-data/crime_09_21_final.csv"
csvfile = pd.read_csv(csvpath,encoding="utf-8")
crime_df = pd.DataFrame(csvfile)
crime_df.head()

C:\Users\joeki\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,offense_id,occur_date,occur_time,location,UC2_Literal,ibr_code,neighborhood,lat,long,timestamp,occur_year,occur_month
0,61681984.0,6/17/2009,1400,375 AUBURN AVE,LARCENY-NON VEHICLE,2308,Browns Mill Park,33.68143,-84.39669,6/17/2009,2009,6
1,72692336.0,2/24/2010,2329,600 MARTIN ST,AGG ASSAULT,1315K,Pittsburgh,33.72212,-84.39460,2/24/2010,2010,2
2,81830088.0,1/25/2010,830,400 FLETCHER ST SW,HOMICIDE,911,Pittsburgh,33.72554,-84.40107,1/25/2010,2010,1
3,90010061.0,1/1/2009,10,501 TUFTON TRL SE,BURGLARY-RESIDENCE,2202,Glenrose Heights,33.66477,-84.38430,1/1/2009,2009,1
4,90010159.0,1/1/2009,57,BUCKEAD AVE/GRANDVIEW AVE,ROBBERY-PEDESTRIAN,1204,Buckhead Village,33.83752,-84.37574,1/1/2009,2009,1


In [25]:
##  Put the months into bins and made "Seasons"
bins = [0,3,6,9,12]
seasons = ["Winter","Spring","Summer","Autumn"]

crime_df["Seasons"] = pd.cut(crime_df["occur_month"],bins,labels=seasons,include_lowest=True, right = False)
crime_df

,offense_id,occur_date,occur_time,location,UC2_Literal,ibr_code,neighborhood,lat,long,timestamp,occur_year,occur_month,Seasons
0,61681984.0,6/17/2009,1400,375 AUBURN AVE,LARCENY-NON VEHICLE,2308,Browns Mill Park,33.681430,-84.396690,6/17/2009,2009,6,Summer
1,72692336.0,2/24/2010,2329,600 MARTIN ST,AGG ASSAULT,1315K,Pittsburgh,33.722120,-84.394600,2/24/2010,2010,2,Winter
2,81830088.0,1/25/2010,830,400 FLETCHER ST SW,HOMICIDE,911,Pittsburgh,33.725540,-84.401070,1/25/2010,2010,1,Winter
3,90010061.0,1/1/2009,10,501 TUFTON TRL SE,BURGLARY-RESIDENCE,2202,Glenrose Heights,33.664770,-84.384300,1/1/2009,2009,1,Winter
4,90010159.0,1/1/2009,57,BUCKEAD AVE/GRANDVIEW AVE,ROBBERY-PEDESTRIAN,1204,Buckhead Village,33.837520,-84.375740,1/1/2009,2009,1,Winter
...,...,...,...,...,...,...,...,...,...,...,...,...,...
345875,210280231.0,1/27/2021,21:15,2343 CAMPBELLTON RD SW,LARCENY-FROM VEHICLE,23F,Southwest,33.704558,-84.465379,1/27/2021,2021,1,Winter
345876,210280239.0,1/28/2021,2:37,485 LINDBERGH PL NE,AGG ASSAULT,13A,Lindbergh/Morosgo,33.820101,-84.371098,1/28/2021,2021,1,Winter
345877,210280257.0,1/28/2021,3:26,3186 US-78,LARCENY-FROM VEHICLE,23F,Collier Heights,33.785756,-84.491515,1/28/2021,2021,1,Winter
345878,210280267.0,1/28/2021,2:23,323 SPRING ST NW,LARCENY-FROM VEHICLE,23F,Downtown,33.763580,-84.388717,1/28/2021,2021,1,Winter


In [37]:
## IDing the different crimes
compartmentalize_crimes = crime_df["UC2_Literal"].unique()
compartmentalize_crimes
print(f"There are {len(compartmentalize_crimes)} different crimes in the 'UC2_Literal' column.")
compartmentalize_crimes

There are 13 different crimes in the 'UC2_Literal' column.


array(['LARCENY-NON VEHICLE', 'AGG ASSAULT', 'HOMICIDE',
       'BURGLARY-RESIDENCE', 'ROBBERY-PEDESTRIAN', 'AUTO THEFT',
       'BURGLARY-NONRES', 'LARCENY-FROM VEHICLE', 'ROBBERY-RESIDENCE',
       'ROBBERY-COMMERCIAL', 'MANSLAUGHTER', 'BURGLARY', 'ROBBERY'],
      dtype=object)

In [42]:
## IDing the different neighborhoods
compartmentalize_hoods = crime_df["neighborhood"].unique()
compartmentalize_hoods
print(f"There are {len(compartmentalize_hoods)} different neighborhoods in the 'neighborhood' column.")
compartmentalize_hoods

There are 243 different neighborhoods in the 'neighborhood' column.


array(['Browns Mill Park', 'Pittsburgh', 'Glenrose Heights',
       'Buckhead Village', 'Oakland City', 'Penelope Neighbors',
       'Adamsville', 'Downtown', 'Westview', 'South Atlanta',
       'Thomasville Heights', 'Peachtree Hills', 'Grove Park',
       'Greenbriar', 'Venetian Hills', 'Pomona Park',
       'The Villages at Carver', 'East Atlanta', 'Candler Park',
       'Bankhead', 'Vine City', 'Old Fourth Ward', 'Peoplestown',
       'Home Park', 'English Avenue', 'Wisteria Gardens',
       'Wilson Mill Meadows', 'Tuxedo Park', 'Lenox', 'Collier Heights',
       'Springlake', 'Edgewood', 'Atlanta Industrial Park',
       'Capitol View', 'Chastain Park', 'West End', 'Mechanicsville',
       'Midtown', 'Lake Claire', 'Lakewood', 'Fairburn Heights',
       'Rosedale Heights', 'Grant Park', 'North Buckhead', 'Adair Park',
       'Lindridge/Martin Manor', 'Scotts Crossing', 'Woodland Hills',
       'Knight Park/Howell Station', 'Sylvan Hills', 'Poncey-Highland',
       'Benteen Park', 

In [41]:
## IDing the different codes
compartmentalize_ibr_codes = crime_df["ibr_code"].unique()
compartmentalize_ibr_codes
print(f"There are {len(compartmentalize_ibr_codes)} different codes in the 'ibr_code' column.")
compartmentalize_ibr_codes

There are 104 different codes in the 'ibr_code' column.


array(['2308', '1315K', '911', '2202', '1204', '2424', '2404', '1399',
       '1314', '2404A', '2399', '2434', '2203', '2303', '2305', '2424A',
       '1299G', '2204', '1315', '1205', '1208K', '2307', '2304', '2302',
       '1205K', '1209', '2317', '1201', '1206', '2301', '2202A', '1207',
       '1318', '1203', '2310', '2203A', '2205', '1299S', '1211S', '2316',
       '2361', '1201D', '1203Y', '1201C', '1208', '2434A', '912', '2373',
       '1202', '2382', '902', '1202R', '1216', '1299O', '2374', '1211G',
       '1202K', '1203X', '1211O', '903', '904', '1299K', '1202L', '2314',
       '1376', '999', '1377', '901', '1375', '1313', '2899', '2902',
       '9999', '2804', '1316', '5707', '2803', '3562', '9920', '5311',
       '7399', '1202Q', '1211K', '5299', '8', '1202J', '2599', '1212',
       '2318', '105', '910', '240', '23C', '23G', '13A', '23H', '220',
       '23F', '120', '23D', '09A', '23A', '23B', '23E'], dtype=object)

In [44]:
homicide_df_df = crime_df.loc[(crime_df["UC2_Literal"]== "HOMICIDE") & (crime_df["Seasons"]== "Summer")]
homicide_df_df

,offense_id,occur_date,occur_time,location,UC2_Literal,ibr_code,neighborhood,lat,long,timestamp,occur_year,occur_month,Seasons
14982,91530164.0,6/2/2009,145,COURTLAND STREET @ PINE STREET,HOMICIDE,911,Downtown,33.76746,-84.38408,6/2/2009,2009,6,Summer
15108,91532225.0,6/2/2009,2214,911 JOSEPH E BOON BLVD NW,HOMICIDE,911,English Avenue,33.76366,-84.41734,6/2/2009,2009,6,Summer
15616,91580138.0,6/7/2009,55,900 NEWTOWN CIRCLE,HOMICIDE,903,Thomasville Heights,33.70837,-84.36007,6/7/2009,2009,6,Summer
15621,91580219.0,6/7/2009,144,1195 WOODLAND AVE NE,HOMICIDE,911,Lindridge/Martin Manor,33.81477,-84.34908,6/7/2009,2009,6,Summer
15868,91592149.0,6/8/2009,2330,1696 EZRA CHURCH DR NW,HOMICIDE,911,West Lake,33.76246,-84.44280,6/8/2009,2009,6,Summer
...,...,...,...,...,...,...,...,...,...,...,...,...,...
320830,192330356.0,8/21/2019,300,1991 DELOWE DR SW,HOMICIDE,911,Campbellton Road,33.70103,-84.45746,8/21/2019,2019,8,Summer
320866,192332299.0,8/21/2019,2015,409 ANDREW J HAIRSTON PL NW,HOMICIDE,911,English Avenue,33.76586,-84.41361,8/21/2019,2019,8,Summer
321288,192400745.0,8/28/2019,939,3000 CONTINENTAL COLONY PKWY SW,HOMICIDE,911,Greenbriar,33.68077,-84.49370,8/28/2019,2019,8,Summer
321353,192410708.0,8/29/2019,900,1666 OLYMPIA WAY SW,HOMICIDE,912,Westview,33.73580,-84.44222,8/29/2019,2019,8,Summer


In [48]:
crime_df_data = crime_df.groupby(['occur_year','Seasons'])
crime_df_data.count().head(50)

offense_id  occur_date  occur_time  location  UC2_Literal  \
occur_year Seasons                                                              
2009       Winter       5878.0      5878.0      5878.0    5878.0       5878.0   
           Spring       9225.0      9225.0      9225.0    9225.0       9225.0   
           Summer      10385.0     10385.0     10385.0   10385.0      10385.0   
           Autumn       9252.0      9252.0      9252.0    9252.0       9252.0   
2010       Winter       4586.0      4586.0      4586.0    4586.0       4586.0   
           Spring       8497.0      8497.0      8497.0    8497.0       8497.0   
           Summer       9237.0      9237.0      9237.0    9237.0       9237.0   
           Autumn       9101.0      9101.0      9101.0    9101.0       9101.0   
2011       Winter       4444.0      4444.0      4444.0    4444.0       4444.0   
           Spring       8283.0      8283.0      8283.0    8283.0       8283.0   
           Summer       9353.0      9353.0      9353.0    9353.0       9353.0   
           Autumn       8560.0      8560.0      8560.0    8560.0       8560.0   
2012       Winter       4552.0      4552.0      4552.0    4552.0       4552.0   
           Spring       7877.0      7877.0      7877.0    7877.0       7877.0   
           Summer       8838.0      8838.0      8838.0    8838.0       8838.0   
           Autumn       8063.0      8063.0      8063.0    8063.0       8063.0   
2013       Winter       4855.0      4855.0      4855.0    4855.0       4855.0   
           Spring       7703.0      7703.0      7703.0    7703.0       7703.0   
           Summer       8503.0      8503.0      8503.0    8503.0       8503.0   
           Autumn       7686.0      7686.0      7686.0    7686.0       7686.0   
2014       Winter       4222.0      4222.0      4222.0    4222.0       4222.0   
           Spring       6961.0      6961.0      6961.0    6961.0       6961.0   
           Summer       8056.0      8056.0      8056.0    8056.0       8056.0   
           Autumn       7968.0      7968.0      7968.0    7968.0       7968.0   
2015       Winter       4227.0      4227.0      4227.0    4227.0       4227.0   
           Spring       7023.0      7023.0      7023.0    7023.0       7023.0   
           Summer       7922.0      7922.0      7922.0    7922.0       7922.0   
           Autumn       7427.0      7427.0      7427.0    7427.0       7427.0   
2016       Winter       4208.0      4208.0      4208.0    4208.0       4208.0   
           Spring       6669.0      6669.0      6669.0    6669.0       6669.0   
           Summer       7356.0      7356.0      7356.0    7356.0       7356.0   
           Autumn       7191.0      7191.0      7191.0    7191.0       7191.0   
2017       Winter       3742.0      3742.0      3742.0    3742.0       3742.0   
           Spring       5718.0      5718.0      5718.0    5718.0       5718.0   
           Summer       6669.0      6669.0      6669.0    6669.0       6669.0   
           Autumn       7033.0      7033.0      7033.0    7033.0       7033.0   
2018       Winter       3821.0      3821.0      3821.0    3821.0       3821.0   
           Spring       5848.0      5848.0      5848.0    5848.0       5848.0   
           Summer       6442.0      6442.0      6442.0    6442.0       6442.0   
           Autumn       6264.0      6264.0      6264.0    6264.0       6264.0   
2019       Winter       3519.0      3519.0      3519.0    3519.0       3519.0   
           Spring       5585.0      5585.0      5585.0    5585.0       5585.0   
           Summer       6453.0      6453.0      6453.0    6453.0       6453.0   
           Autumn       6241.0      6241.0      6241.0    6241.0       6241.0   
2020       Winter         18.0        18.0        18.0      18.0         18.0   
           Spring          5.0         5.0         5.0       5.0          5.0   
           Summer         51.0        51.0        51.0      51.0         51.0   
           Autumn       9335.0      9335.0      933

In [24]:
crime_df_data = crime_df.groupby(['UC2_Literal','occur_year','Seasons'])
crime_df_data.count().head(50)

offense_id  occur_date  occur_time  location  \
UC2_Literal occur_year Seasons                                                 
AGG ASSAULT 2009       Winter        404.0       404.0       404.0     404.0   
                       Spring        658.0       658.0       658.0     658.0   
                       Summer        687.0       687.0       687.0     687.0   
                       Autumn        581.0       581.0       581.0     581.0   
            2010       Winter        329.0       329.0       329.0     329.0   
                       Spring        659.0       659.0       659.0     659.0   
                       Summer        680.0       680.0       680.0     680.0   
                       Autumn        655.0       655.0       655.0     655.0   
            2011       Winter        332.0       332.0       332.0     332.0   
                       Spring        642.0       642.0       642.0     642.0   
                       Summer        667.0       667.0       667.0     667.0   
                       Autumn        600.0       600.0       600.0     600.0   
            2012       Winter        332.0       332.0       332.0     332.0   
                       Spring        714.0       714.0       714.0     714.0   
                       Summer        650.0       650.0       650.0     650.0   
                       Autumn        540.0       540.0       540.0     540.0   
            2013       Winter        320.0       320.0       320.0     320.0   
                       Spring        556.0       556.0       556.0     556.0   
                       Summer        587.0       587.0       587.0     587.0   
                       Autumn        522.0       522.0       522.0     522.0   
            2014       Winter        286.0       286.0       286.0     286.0   
                       Spring        543.0       543.0       543.0     543.0   
                       Summer        593.0       593.0       593.0     593.0   
                       Autumn        520.0       520.0       520.0     520.0   
            2015       Winter        279.0       279.0       279.0     279.0   
                       Spring        544.0       544.0       544.0     544.0   
                       Summer        599.0       599.0       599.0     599.0   
                       Autumn        478.0       478.0       478.0     478.0   
            2016       Winter        268.0       268.0       268.0     268.0   
                       Spring        520.0       520.0       520.0     520.0   
                       Summer        612.0       612.0       612.0     612.0   
                       Autumn        537.0       537.0       537.0     537.0   
            2017       Winter        261.0       261.0       261.0     261.0   
                       Spring        516.0       516.0       516.0     516.0   
                       Summer        543.0       543.0       543.0     543.0   
                       Autumn        495.0       495.0       495.0     495.0   
            2018       Winter        249.0       249.0       249.0     249.0   
                       Spring        486.0       486.0       486.0     486.0   
                       Summer        466.0       466.0       466.0     466.0   
                       Autumn        427.0       427.0       427.0     427.0   
            2019       Winter        240.0       240.0       240.0     240.0   
                       Spring        518.0       518.0       518.0     518.0   
                       Summer        537.0       537.0       537.0     537.0   
                       Autumn        464.0       464.0       464.0     464.0   
            2020       Winter          1.0         1.0         1.0       1.0   
                       Spring          NaN         NaN         NaN       NaN   
                       Summer          NaN         NaN         NaN       NaN   
                       Autumn        911.0       911.0       911.0     911.0   
            2021       Winter   